# Data Import from drive

In [1]:
# %cd ..
# %pwd
# !cp '/content/drive/My Drive/IEEE BigMM/ieee-bigmm-images.zip' './'
!git clone 'https://github.com/sohamtiwari3120/ieee-bigmm-images.git'

Cloning into 'ieee-bigmm-images'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 7175 (delta 12), reused 8 (delta 3), pack-reused 7142
Receiving objects: 100% (7175/7175), 592.44 MiB | 42.36 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Checking out files: 100% (8551/8551), done.


In [2]:
%ls

ieee-bigmm-images/  sample_data/


In [3]:
# %%capture
# !unzip ieee-bigmm-images.zip

In [4]:
%cd ieee-bigmm-images/

/content/ieee-bigmm-images


In [5]:
!git pull origin master

From https://github.com/sohamtiwari3120/ieee-bigmm-images
 * branch            master     -> FETCH_HEAD
Already up to date.


In [6]:
%ls

clean_datav5.csv                README.md
clean_datav6.csv                test_data_cleaned.csv
Data_without-invalid_cells.csv  test_images/
final_dataset.csv               test_tweet_2.csv
final_test2.csv                 train_images/
final_test3_unpreprocessed.csv


# Code

In [7]:
# %%capture
!pip install pytorch_pretrained_bert
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
# !pip3 install torchvision
! pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install imbalanced-learn

     |████████████████████████████████| 133kB 4.6MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 25kB/s 
     |████████████████████████████████| 6.6MB 61.0MB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [8]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import os
from pytorch_pretrained_bert.modeling import BertModel
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertAdam
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import tqdm
import datetime
import random

In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [10]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
print(device)
print(torch.cuda.is_available())

cuda
True


In [11]:
df = pd.read_csv('./clean_datav6.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,tweet_id,text,missing_text,Text_Only_Informative,Image_Only_Informative,Directed_Hate,Generalized_Hate,Sarcasm,Allegation,Justification,Refutation,Support,Oppose
0,0,0,0,1052237153789390853,New post (Domestic Violence Awareness Hasn't C...,0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1,1,1,1052207832081129472,Domestic Violence Awareness Hasn’t Caught Up W...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2,2,1052183746344960000,Mother Nature’s #MeToo,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,3,1052156864840908800,ption - no:2,0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,4,4,1052095305133510656,It is 'high time' #MeToo named and shamed men ...,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [12]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, train_size=0.8, shuffle = True )
train_df = train_df.reset_index()
val_df = val_df.reset_index()
train_df['text'].head()

0    Clean bowled by Congress, Now this is another ...
1    @jbowser74 @rgrun1 @Michell39030764 @NelsonFor...
2    #MeToo: Raveena Tandon talks about workplace h...
3    I am at the @ekpotliretki public hearing in #C...
4    If you missed it, check out Cheryl Mcwhorter a...
Name: text, dtype: object

In [13]:
img_transformations = transforms.Compose(
        [
            transforms.Resize(256),
#             transforms.Resize((224, 244)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.46777044, 0.44531429, 0.40661017],
                std=[0.12221994, 0.12145835, 0.14380469],
            ),
        ]
    )

In [14]:
bert = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 407873900/407873900 [00:13<00:00, 29964157.16B/s]


In [15]:
bert_tokenizer = BertTokenizer.from_pretrained(
            'bert-base-uncased', do_lower_case=True
        )

100%|██████████| 231508/231508 [00:00<00:00, 874856.01B/s]


In [16]:
(bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize('new post domestic violence awareness caught me zzzzzx83272@xxxx')))

[2047,
 2695,
 4968,
 4808,
 7073,
 3236,
 2033,
 1062,
 13213,
 13213,
 2595,
 2620,
 16703,
 2581,
 2475,
 1030,
 22038,
 20348]

In [17]:
bert_tokenizer.convert_tokens_to_ids(["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

[0, 100, 101, 102, 103]

In [18]:
# help(bert)
# Help on BertModel in module pytorch_pretrained_bert.modeling object:

# class BertModel(BertPreTrainedModel)
#  |  BERT model ("Bidirectional Embedding Representations from a Transformer").
#  |  
#  |  Params:
#  |      config: a BertConfig class instance with the configuration to build a new model
#  |  
#  |  Inputs:
#  |      `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
#  |          with the word token indices in the vocabulary(see the tokens preprocessing logic in the scripts
#  |          `extract_features.py`, `run_classifier.py` and `run_squad.py`)
#  |      `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
#  |          types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
#  |          a `sentence B` token (see BERT paper for more details).
#  |      `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
#  |          selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
#  |          input sequence length in the current batch. It's the mask that we typically use for attention when
#  |          a batch has varying length sentences.
#  |      `output_all_encoded_layers`: boolean which controls the content of the `encoded_layers` output as described below. Default: `True`.
#  |  
#  |  Outputs: Tuple of (encoded_layers, pooled_output)
#  |      `encoded_layers`: controled by `output_all_encoded_layers` argument:
#  |          - `output_all_encoded_layers=True`: outputs a list of the full sequences of encoded-hidden-states at the end
#  |              of each attention block (i.e. 12 full sequences for BERT-base, 24 for BERT-large), each
#  |              encoded-hidden-state is a torch.FloatTensor of size [batch_size, sequence_length, hidden_size],
#  |          - `output_all_encoded_layers=False`: outputs only the full sequence of hidden-states corresponding
#  |              to the last attention block of shape [batch_size, sequence_length, hidden_size],
#  |      `pooled_output`: a torch.FloatTensor of size [batch_size, hidden_size] which is the output of a
#  |          classifier pretrained on top of the hidden state associated to the first character of the
#  |          input (`CLS`) to train on the Next-Sentence task (see BERT's paper). 


In [19]:
# help(bert.encoder)

In [20]:
class TextNImageDataset(Dataset):
    def __init__(self, data, image_path, label_name, transforms, tokenizer, vocab, minority_class):
        self.data = data
        self.image_path = (image_path)
        self.label_name = label_name
        self.transforms = transforms
        self.tokenizer = tokenizer
        self.max_sent_len = 128 - 7 - 2 #since there will be [CLS] <7 image embeddings> [SEP] <the text embeddings>
        self.vocab = vocab
        df2 = self.data[self.data[label_name]==minority_class]
        df2 = df2.copy().reset_index(drop=True)
        df3 = df2.copy().reset_index(drop=True)
        df4 = df2.copy().reset_index(drop=True)
        df5 = df2.copy().reset_index(drop=True)
        # print(df2)
        print(f"Old data length : {len(self.data)}")
        print(f'minority class is {minority_class}. Duplicating minority class data!')
        for i in range(len(df2)):
            text = df2['text'][i]
            text = text.split(' ')
            random.shuffle(text)
            text2 = ' '.join(text)
            df2['text'][i]=text2
            random.shuffle(text)
            text3 = ' '.join(text)
            df3['text'][i]=text3
            random.shuffle(text)
            text4 = ' '.join(text)
            df4['text'][i]=text4
            random.shuffle(text)
            text5 = ' '.join(text)
            df5['text'][i]=text5
        #self.data = self.data.append(df2, ignore_index=True)
        #self.data = self.data.append(df3, ignore_index=True)
        #self.data = self.data.append(df4, ignore_index=True)
        #self.data = self.data.append(df5, ignore_index=True)
        #self.data = self.data.reset_index(drop=True)
        print(f"New data length : {len(self.data)}")

    def __getitem__(self,  index):
        text = self.data['text'][index]
        text = str(text)
        text = self.tokenizer.tokenize(text)[:self.max_sent_len]
        text = torch.LongTensor(self.tokenizer.convert_tokens_to_ids(text))
        tweet_id = self.data['tweet_id'][index]
        label = torch.LongTensor([self.data[self.label_name][index]])
        image = None
        try:
            image = Image.open(
                self.image_path+"/"+str(tweet_id)+".jpg"
            ).convert("RGB")
#             print(self.image_path+"/"+str(tweet_id)+".jpg"+" opened!")
#             image.show()
            image = self.transforms(image)
        except:
            image = Image.fromarray(128*np.ones((256, 256, 3), dtype=np.uint8))
            image = self.transforms(image)
            
        return text, label, image
    
    def __len__(self):
        return len(self.data)


class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder, self).__init__()
        model = torchvision.models.resnet152(pretrained=True)
        modules = list(model.children())[:-2]
        # we are removing the last adaptive average pooling layer and the 
        # the classification layer
        self.model = nn.Sequential(*modules)
        if(torch.cuda.is_available()):
            self.model = self.model.cuda()
        # self.model = self.model.to(device)

    def forward(self, x):
        out = (self.model(x))
        # print('Model output', out.size())

        out = nn.AdaptiveAvgPool2d((7, 1))(out)#specifying the H and W of the image
        # to be obtained after pooling
        # print('Pooling output', out.size())

        out = torch.flatten(out, start_dim=2)
        # print('Flattening output', out.size())

        out = out.transpose(1, 2).contiguous()
        # print('Transpose output', out.size())
        
        return out


class Vocab(object):
    def __init__(self, emptyInit=False):
        if emptyInit:
            self.stoi={}#string to index dictionary
            self.itos=[]#index to string dictionary
            self.vocab_size=0
        else:
            self.stoi={
                w:i
                for i, w in enumerate(["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
            }
            self.itos = [w for w in self.stoi]
            self.vocab_size = len(self.itos)
    
    def add(self, words):
        counter = len(self.itos)
        for w in words:
            if w in self.stoi:
                continue
            self.stoi[w]=counter
            counter+=1
            self.itos.append(w)
        self.vocab_size = len(self.itos)

class ImageEmbeddingsForBert(nn.Module):
    def __init__(self, embeddings, vocabObject):
        super(ImageEmbeddingsForBert, self).__init__()
        self.vocab = vocabObject
#       the embeddins received as input are the 
#       all the embeddings provided by the bert model from pytorch
        self.img_embeddings = nn.Linear(2048, 768)
#       above is linear layer is used to convert the flattened images 
#       logits obtained after pooling from Image encoder which have 2048
#       dimensions to a 768 dimensions which is the size of bert's hidden layer
        
        self.position_embeddings = embeddings.position_embeddings
        self.token_type_embeddings = embeddings.token_type_embeddings
        self.word_embeddings = embeddings.word_embeddings
        self.LayerNorm = embeddings.LayerNorm
        self.dropout = embeddings.dropout
        
    def forward(self, batch_input_imgs, token_type_ids):
        batch_size = batch_input_imgs.size(0)
        seq_length = 7 + 2
#         since we are assuming that from each image we will obtain
#         7 image embeddings of 768 dimensions each
        
        cls_id = torch.LongTensor([101])
        if torch.cuda.is_available():
            cls_id = cls_id.cuda()
            self.word_embeddings = self.word_embeddings.cuda()
        cls_id = cls_id.unsqueeze(0).expand(batch_size, 1)
        
        if torch.cuda.is_available():
            cls_id = cls_id.cuda()
        cls_token_embeddings = self.word_embeddings(cls_id)
        
        sep_id = torch.LongTensor([102])
        if torch.cuda.is_available():
            sep_id = sep_id.cuda()
            self.img_embeddings = self.img_embeddings.cuda()
        sep_id = sep_id.unsqueeze(0).expand(batch_size, 1)
        sep_token_embeddings = self.word_embeddings(sep_id)
        
        batch_image_embeddings_768 = self.img_embeddings(batch_input_imgs)
        
        token_embeddings = torch.cat(
        [cls_token_embeddings, batch_image_embeddings_768, sep_token_embeddings], dim=1)
        
        position_ids = torch.arange(seq_length, dtype=torch.long)
        if torch.cuda.is_available():
            position_ids = position_ids.cuda()
            self.position_embeddings = self.position_embeddings.cuda()
            self.token_type_embeddings= self.token_type_embeddings.cuda()
        position_ids = position_ids.unsqueeze(0).expand(batch_size, seq_length)
        
        position_embeddings = self.position_embeddings(position_ids)
        
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        
        embeddings = token_embeddings+position_embeddings+token_type_embeddings
        if torch.cuda.is_available():
            embeddings = embeddings.cuda()
            self.LayerNorm=self.LayerNorm.cuda()
            self.dropout=self.dropout.cuda()
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        
        return embeddings


class MultiModalBertEncoder(nn.Module):
    def __init__(self, no_of_classes, tokenizer):
        super(MultiModalBertEncoder, self).__init__()
        bert = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = tokenizer
        self.embeddings = bert.embeddings
        self.vocab=Vocab()
        self.image_embeddings = ImageEmbeddingsForBert(self.embeddings, self.vocab)
        self.image_encoder = ImageEncoder()
        self.encoder = bert.encoder
        self.pooler = bert.pooler
        self.clf = nn.Linear(768, no_of_classes)
        
    def forward(self, input_text, text_attention_mask, text_segment, input_image):
        batch_size = input_text.size(0)
# input text is a tensor of encoded texts!
        temp = torch.ones(batch_size, 7+2).long()
        if torch.cuda.is_available():
            temp = temp.cuda()
            self.encoder = self.encoder.cuda()
            self.pooler = self.pooler.cuda()
        attention_mask = torch.cat(
            [
                temp, text_attention_mask
            ],
            dim=1
        )
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
#         print(attention_mask.shape, extended_attention_mask.shape)
        extended_attention_mask = extended_attention_mask.to(
            dtype=next(self.parameters()).dtype
        )
        # extended_attention_mask = (1.0 - extended_attention_mask)*-10000.0
        
        image_token_type_ids = torch.LongTensor(batch_size, 7+2).fill_(0)
        if(torch.cuda.is_available()):
            image_token_type_ids= image_token_type_ids.cuda()
        
        image = self.image_encoder(input_image)
#         above image returned is of the formc nC x nH x nW and is a tensor
        image_embedding_out = self.image_embeddings(image, image_token_type_ids)
#         print('Image embeddings: ', image_embedding_out.size())
        
        text_embedding_out = self.embeddings(input_text, text_segment)
#         print('Text embeddings: ', text_embedding_out.size(), text_embedding_out)
#         print(input_text, text_embedding_out)
        
        encoder_input = torch.cat([image_embedding_out, text_embedding_out], dim=1)
#         the encoder input is of the form CLS (7 image embeddings) SEP text_embeddings
    
        encoded_layers = self.encoder(encoder_input, extended_attention_mask, output_all_encoded_layers=False)
        # above function returns the hidden states off all the layers L in the bert model. in case of bert base, L = 12;
        # if output all encoded layers is false, then only returns the hidden state of the last self attention layer
        # print('ENCODED_LAYERS',encoded_layers[-1],'enc layers2', encoded_layers[-1][:][0])
        final = self.pooler(encoded_layers[-1])
        # print('FINAL POOLED LAYERS', final, final.size())
#         print('encoded layers', encoded_layers)
        return final
        # how to extract CLS layer
        

class MultiModalBertClf(nn.Module):
    def __init__(self, no_of_classes, tokenizer):
        super(MultiModalBertClf, self).__init__()
        self.no_of_classes = no_of_classes
        self.enc = MultiModalBertEncoder(self.no_of_classes, tokenizer)
        # self.layer1 = nn.Linear(768, 512)
        # self.layer2 = nn.Linear(512, 256)
        self.batch_norm = nn.BatchNorm1d(768)
        self.clf = nn.Linear(768, self.no_of_classes)
    
    def forward(self, text, text_attention_mask, text_segment, image):
        if(torch.cuda.is_available()):
            text = text.cuda()
            text_attention_mask=text_attention_mask.cuda()
            text_segment=text_segment.cuda()
            image = image.cuda()
            self.clf = self.clf.cuda()
        x = self.enc(text, text_attention_mask, text_segment, image)
        # x = F.relu(self.layer1(x))
        # x = F.relu(self.layer2(x))
        x = self.batch_norm(x)
        x = self.clf(x)
        # print('Sigmoid output: ',torch.sigmoid(x))
        return x 


class FocalLoss(nn.Module):
    # read the focal loss paper
    def __init__(self, alpha=1, gamma=2, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce
        
    def forward(self, y_pred, y_true):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(y_pred.squeeze(-1), y_true.squeeze(-1), reduce = None)#this automatically  takes sigmoid of logits
        else:
            BCE_loss = F.binary_cross_entropy(y_pred, y_true, reduce = None)
            
        pt = torch.exp(-BCE_loss)
#       # pt = p if y = 1
#       # pt = 1 - p if y = else
#       p is the predicted value, y is the target label
        # pt is used to indicate if the prediction matches the target or not
        # if pt->1, then proper classification, else if pt->0, then misclassification
        # so focal loss basically downweights the loss generated in a proper classification
        # but does not change downweight the loss in a miss classification
        F_loss =self.alpha * ((1-pt)**self.gamma) * BCE_loss
        if self.reduce:
            return torch.mean(F_loss)
        return F_loss
        
class DiceLoss(nn.Module):
    def __init__(self, logits = True):
        super(DiceLoss, self).__init__()

    def forward(self, y_pred, y_true, logits=True, smooth=1):
        if(logits):
            y_pred = torch.sigmoid(y_pred)
        y_pred = y_pred.view(-1)
        y_true = y_true.view(-1)

        intersection = (y_pred*y_true).sum()
        pred_sum = (y_pred*y_pred).sum()
        true_sum = (y_true*y_true).sum()

        return 1 - (2 * intersection + smooth) / (pred_sum + true_sum+smooth)

In [21]:
def collate_function_for_dataloader(batch, task_type='singlelabel'):
    lengths = [len(row[0]) for row in batch]
    batch_size = len(batch)
    max_sent_len = max(lengths)
    if(max_sent_len>128-7-2):
        max_sent_len=128-7-2
    text_tensors = torch.zeros(batch_size, max_sent_len).long()
    text_attention_mask = torch.zeros(batch_size, max_sent_len).long()
    text_segment = torch.zeros(batch_size, max_sent_len).long()
    
    batch_image_tensors = torch.stack([row[2] for row in batch])
    
    label_tensors = torch.cat([row[1] for row in batch]).long()
    if task_type=='multilabel':
        label_tensors = torch.stack([row[1] for row in batch])
#     note there is a difference between stack and cat, refer link below if needed
# https://stackoverflow.com/questions/54307225/whats-the-difference-between-torch-stack-and-torch-cat-functions
    
    for i, (row, length) in enumerate(zip(batch, lengths)):
        text_tokens = row[0]
        if(length>128-7-2):
            length = 128-7-2
        text_tensors[i, :length] = text_tokens
        text_segment[i, :length] = 1#since images will constitute segment 0
        text_attention_mask[i, :length]=1
    
    return text_tensors, label_tensors, text_segment, text_attention_mask, batch_image_tensors


def get_optimizer(model, train_data_len, batch_size = 4, gradient_accumulation_steps=1, max_epochs=3, lr=0.001):
    total_steps = (
        train_data_len
        / batch_size
        / gradient_accumulation_steps
        * max_epochs
    )
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
        {"params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], "weight_decay": 0.0,},
    ]
    # print('OPTIMIZER PARAMS', optimizer_grouped_parameters)
    optimizer = BertAdam(
        optimizer_grouped_parameters,
        lr=lr,
#         warmup=args.warmup,
        t_total=total_steps,
    )
#     optimizer = optim.Adam(
#         optimizer_grouped_parameters,
#         lr=lr,
# #         warmup=args.warmup,
#         t_total=total_steps,
#     )
    return optimizer

def model_forward(i_epoch, model, criterion, batch):
    txt, tgt, segment, mask, img= batch

#         for param in model.enc.img_encoder.parameters():
#             param.requires_grad = not freeze_img
#         for param in model.enc.encoder.parameters():
#             param.requires_grad = not freeze_txt
    if(torch.cuda.is_available()):
        txt, img = txt.cuda(), img.cuda()
        mask, segment = mask.cuda(), segment.cuda()
    out = model(txt, mask, segment, img)
    if(torch.cuda.is_available()):
        tgt = tgt.cuda()
    # print()
    loss = criterion(out*1.0, tgt.unsqueeze(1)*1.0)
    return loss, out, tgt


def store_preds_to_disk(tgts, preds, savedir):
    str_time = str(datetime.datetime.now())
    with open(os.path.join(savedir, "./test_labels_pred_"+str_time+"_.txt"), "w") as fw:
        fw.write("\n".join([str(x) for x in preds]))
    with open(os.path.join(savedir, "./test_labels_actual_"+str_time+"_.txt"), "w") as fw:
        fw.write("\n".join([str(x) for x in tgts]))
#     with open(os.path.join(savedir, "test_labels.txt"), "w") as fw:
#         fw.write(" ".join([str(l) for l in alabels]))


def model_eval(i_epoch, data, model, criterion, no_of_classes, store_preds=False):
    with torch.no_grad():
        losses, preds, tgts = [], [], []
        for batch in data:
            loss, out, tgt = model_forward(i_epoch, model, criterion, batch)
            losses.append(loss.item())
            if no_of_classes==1:
                pred = torch.sigmoid(out).cpu().detach().numpy()
                # for i in range(pred.shape[0]):
                #     if pred[i][0]>0.5:
                #         pred[i][0]=1
                #     else:
                #         pred[i][0]=0
            else:
                pred = torch.nn.functional.softmax(out, dim=1).argmax(dim=1).cpu().detach().numpy()
                
            preds.append(pred)
            tgt = tgt.cpu().detach().numpy()
            tgts.append(tgt)

    metrics = {"loss": np.mean(losses)}
    tgts = [l for sl in tgts for l in sl]
    preds = [l for sl in preds for l in sl]
    # metrics["acc"] = accuracy_score(tgts, preds)
    # metrics['classification_report'] = classification_report(tgts, preds)
    metrics['roc_auc_score'] = roc_auc_score(tgts, preds)

    if store_preds:
        store_preds_to_disk(tgts, preds, './')

    return metrics

In [22]:
class SubmissionDataset(Dataset):
    def __init__(self, data, image_path, transforms, tokenizer, vocab):
        self.data = data
        self.image_path = (image_path)
        self.transforms = transforms
        self.tokenizer = tokenizer
        self.max_sent_len = 128 - 7 - 2 #since there will be [CLS] <7 image embeddings> [SEP] <the text embeddings>
        self.vocab = vocab
    def __getitem__(self,  index):
        text = self.data['text'][index]
        text = str(text)
        text = self.tokenizer.tokenize(text)[:self.max_sent_len]
        text = torch.LongTensor(self.tokenizer.convert_tokens_to_ids(text))
        tweet_id = self.data['TweetId'][index]
#         label = torch.LongTensor([self.data[self.label_name][index]])
        image = None
        try:
            image = Image.open(
                self.image_path+"/"+str(tweet_id)+".jpg"
            ).convert("RGB")
#             print(self.image_path+"/"+str(tweet_id)+".jpg"+" opened!")
#             image.show()
            image = self.transforms(image)
        except:
            image = Image.fromarray(128*np.ones((256, 256, 3), dtype=np.uint8))
            image = self.transforms(image)
            
        return text, image, tweet_id
    
    def __len__(self):
        return len(self.data)


def collate_function_for_submission(batch, task_type='singlelabel'):
    lengths = [len(row[0]) for row in batch]
    batch_size = len(batch)
    max_sent_len = max(lengths)
    if(max_sent_len>128-7-2):
        max_sent_len=128-7-2
    text_tensors = torch.zeros(batch_size, max_sent_len).long()
    text_attention_mask = torch.zeros(batch_size, max_sent_len).long()
    text_segment = torch.zeros(batch_size, max_sent_len).long()
    batch_image_tensors = torch.stack([row[1] for row in batch])
    tweet_id_tensors = torch.zeros(batch_size, 1).long()
    
    # label_tensors = torch.cat([row[1] for row in batch]).long()
    # if task_type=='multilabel':
        # label_tensors = torch.stack([row[1] for row in batch])
#     note there is a difference between stack and cat, refer link below if needed
# https://stackoverflow.com/questions/54307225/whats-the-difference-between-torch-stack-and-torch-cat-functions
    
    for i, (row, length) in enumerate(zip(batch, lengths)):
        text_tokens = row[0]
        if(length>128-7-2):
            length = 128-7-2
        text_tensors[i, :length] = text_tokens
        text_segment[i, :length] = 1#since images will constitute segment 0
        text_attention_mask[i, :length]=1
        tweet_id_tensors[i, 0]=row[2]
    
    return text_tensors, text_segment, text_attention_mask, batch_image_tensors, tweet_id_tensors

In [23]:
def train(label_name, no_of_classes, max_epochs, train_df, val_df, img_transformations, bert_tokenizer, vocab, gradient_accumulation_steps=1, patience=0):
    
    train_dataset = TextNImageDataset(train_df, './train_images', label_name, img_transformations, bert_tokenizer, vocab, minority_class)
    val_dataset = TextNImageDataset(val_df, './train_images', label_name, img_transformations, bert_tokenizer, vocab, minority_class)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_function_for_dataloader, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, collate_fn=collate_function_for_dataloader, drop_last=True)

    model = MultiModalBertClf(no_of_classes, bert_tokenizer)
    try:
        model.load_state_dict(torch.load('./model_state_dict.pth'))
        print('Loaded previous model state successfully!')
    except:
        print('Starting fresh! Previous model state dict load unsuccessful')
    criterion = nn.CrossEntropyLoss()
    if no_of_classes==1:
        print('using '+str(chosen_criteria)+' loss')
        criterion = chosen_criteria
    optimizer = get_optimizer(model, train_dataset.__len__(), max_epochs=max_epochs, gradient_accumulation_steps=gradient_accumulation_steps)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "max", 
        patience=patience, 
        verbose=True, 
#         factor=args.lr_factor
    )
    if(torch.cuda.is_available()):
        model=model.cuda()


    start_epoch, global_step, n_no_improve, best_metric = 0, 0, 0, -np.inf

    print("Training..")
    for i_epoch in range(start_epoch, max_epochs):
        train_losses = []
        model.train()
        optimizer.zero_grad()

        for batch in tqdm.notebook.tqdm(train_loader, total=len(train_loader)):
            loss, _, _ = model_forward(i_epoch, model, criterion, batch)
            # if gradient_accumulation_steps > 1:
            #     loss = loss / gradient_accumulation_steps

            train_losses.append(loss.item())
            loss.backward()
            global_step += 1
            if global_step % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

        model.eval()
        metrics = model_eval(i_epoch, val_loader, model, criterion, no_of_classes, True)
        print("Train Loss: {:.4f}".format(np.mean(train_losses)))
        print('Train Losses :', train_losses)
        print("Val loss", metrics['loss'])
        # print(metrics['acc'])
        # print(metrics['classification_report'])
        print('Val auc roc', metrics['roc_auc_score'])
        tuning_metric = ( metrics['roc_auc_score'])
        scheduler.step(tuning_metric)
        is_improvement = tuning_metric > best_metric
        if is_improvement:
            best_metric = tuning_metric
            n_no_improve = 0
        else:
            n_no_improve += 1
        
        torch.save(model.state_dict(), './model_state_dict.pth')
        print(f'Saved model state dict for epoch {i_epoch} ')
        # if n_no_improve >= patience:
        #     print("No improvement. Breaking out of loop.")
        #     break

#     load_checkpoint(model, os.path.join(args.savedir, "model_best.pt"))
#     model.eval()
# #     for test_name, test_loader in test_loaders.items():
#     test_metrics = model_eval(
#         np.inf, val_loader, model, criterion, no_of_classes, store_preds=True
#     )
#     print(f"Test - ", test_metrics['loss'])
#     print(test_metrics['acc'])
#     print(test_metrics['classification_report'])
#     print(test_metrics['roc_auc_score'])

#     torch.save(model.state_dict(), './modelv1.pth')
    return model
    # return model, test_metrics


def model_forward_predict(i_epoch, model, criterion, batch):
    txt, segment, mask, img, tweet_id= batch

#         for param in model.enc.img_encoder.parameters():
#             param.requires_grad = not freeze_img
#         for param in model.enc.encoder.parameters():
#             param.requires_grad = not freeze_txt
    if(torch.cuda.is_available()):
        txt, img = txt.cuda(), img.cuda()
        mask, segment = mask.cuda(), segment.cuda()
    out = model(txt, mask, segment, img)
    # if(torch.cuda.is_available()):
    #     tgt = tgt.cuda()
    # loss = criterion(out*1.0, tgt.unsqueeze(1)*1.0)
    return out, tweet_id


def model_predict(dataloader, model, criterion, no_of_classes, store_preds=False):
    with torch.no_grad():
        losses, preds, tgts, tweet_ids = [], [], [], []
        for batch in dataloader:
            out, tweet_id = model_forward_predict(1, model, criterion, batch)
            # losses.append(loss.item())
            if no_of_classes==1:
                pred = torch.sigmoid(out).cpu().detach().numpy()
                # for i in range(pred.shape[0]):
                #     if pred[i][0]>0.5:
                #         pred[i][0]=1
                #     else:
                #         pred[i][0]=0
            else:
                pred = torch.nn.functional.softmax(out, dim=1).argmax(dim=1).cpu().detach().numpy()
            # for i in range(4):
            #     if(pred[i])
            
            # print('preddhd', pred)
            # if pred > 0.5:
            #     preds.append(1)
            # else:
            #     preds.append(0)

            preds.append(pred)
            # tgt = tgt.cpu().detach().numpy()
            # tgts.append(tgt)
            tweet_id = tweet_id.cpu().detach().numpy()
            tweet_ids.append(tweet_id)

    # metrics = {"loss": np.mean(losses)}
    # tgts = [l for sl in tgts for l in sl]
    preds = [l for sl in preds for l in sl]
    # for i in len(preds):
    #     if preds[i]>0.5:
    #         preds[i]=1
    #     else:
    #         preds[i]=0
    tweet_ids = [l for sl in tweet_ids for l in sl]
    # metrics["acc"] = accuracy_score(tgts, preds)
    # metrics['classification_report'] = classification_report(tgts, preds)
    # metrics['roc_auc_score'] = roc_auc_score(tgts, preds)

    # if store_preds:
    #     store_preds_to_disk(tweet_ids, preds, './')

    return preds, tweet_ids

In [24]:
col_name = "Refutation"
train_epochs = 3
losses = [FocalLoss, DiceLoss, nn.BCEWithLogitsLoss]
chosen_criteria = losses[0]()
no_of_classes = 1
print(str(chosen_criteria))
minority_class = 1 # or 0

FocalLoss()


In [25]:
vocab = Vocab()

In [26]:
model = train(col_name, no_of_classes, train_epochs, train_df , val_df, img_transformations, bert_tokenizer, vocab)

Old data length : 6382
minority class is 1. Duplicating minority class data!
New data length : 6382
Old data length : 1596
minority class is 1. Duplicating minority class data!
New data length : 1596


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to b


Starting fresh! Previous model state dict load unsuccessful
using FocalLoss() loss
Training..


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)



Train Loss: 0.0288
Train Losses : [0.18342357873916626, 0.41181591153144836, 1.6242187023162842, 2.205982208251953, 2.0727367401123047, 2.044450283050537, 1.1804708242416382, 0.3109275996685028, 0.5567217469215393, 0.49198096990585327, 0.16063381731510162, 0.23438642919063568, 0.6792712211608887, 1.5888301134109497, 1.4611544609069824, 1.1048426628112793, 0.778170645236969, 0.8594663739204407, 0.31049177050590515, 0.21369709074497223, 0.18129420280456543, 0.0682048350572586, 0.07820187509059906, 0.0417073518037796, 0.07380183041095734, 0.1575264036655426, 0.04321425035595894, 0.01418489869683981, 0.0385943278670311, 0.021806571632623672, 0.017552899196743965, 0.006769451778382063, 0.022145984694361687, 0.0030613865237683058, 0.002674646209925413, 0.005523490719497204, 0.2659463882446289, 0.005433239042758942, 0.0025521297939121723, 0.002167836530134082, 0.009319360367953777, 0.0034400750882923603, 0.0029899636283516884, 0.001500039012171328, 0.001395170809701085, 0.0009965032804757357


Train Loss: 0.0152
Train Losses : [0.0031022033654153347, 0.0030834118369966745, 0.0029113891068845987, 0.0026726392097771168, 0.004957797937095165, 0.0037795696407556534, 0.003001550678163767, 0.003889505285769701, 0.0024313009344041348, 0.0028023819904774427, 0.002309320727363229, 0.003715147264301777, 0.003911200445145369, 0.0026777430903166533, 0.0035885248798877, 0.0019947357941418886, 0.0031712527852505445, 0.0027645931113511324, 0.002354108728468418, 0.00206456589512527, 0.0018275639740750194, 0.07603718340396881, 0.002884741872549057, 0.0019496340537443757, 0.0024893644731491804, 0.0021196005400270224, 0.0017420081421732903, 0.0017799633787944913, 0.0018825622973963618, 0.002088759560137987, 0.002141344826668501, 0.0029749602545052767, 0.0016350819496437907, 0.003542947815731168, 0.0017201144946739078, 0.002900021616369486, 0.001519100391305983, 0.001477035111747682, 0.001285939710214734, 0.0018918064888566732, 0.002102762693539262, 0.0015326767461374402, 0.0018290551379323006


Train Loss: 0.0146
Train Losses : [0.0027520337607711554, 0.20143026113510132, 0.0026441027875989676, 0.0025773330125957727, 0.0025896287988871336, 0.0025569340214133263, 0.0025615841150283813, 0.1460648626089096, 0.0026604279410094023, 0.0027848437894135714, 0.0026818697806447744, 0.0026947397273033857, 0.0026821817737072706, 0.0026091800536960363, 0.0025211439933627844, 0.0027783263940364122, 0.12347409874200821, 0.0026125155854970217, 0.0025794128887355328, 0.002594463061541319, 0.0025270753540098667, 0.0025222913827747107, 0.0027195161674171686, 0.002514549298211932, 0.002604852896183729, 0.002549818018451333, 0.002608052222058177, 0.0026041141245514154, 0.0026373579166829586, 0.002543346956372261, 0.0025393208488821983, 0.002510974882170558, 0.0025926134549081326, 0.0026019755750894547, 0.00277169537730515, 0.0026275671552866697, 0.002541003981605172, 0.0025906278751790524, 0.002508762525394559, 0.0025049420073628426, 0.0026961606927216053, 0.002560435328632593, 0.002558229723945

In [27]:
model = MultiModalBertClf(no_of_classes, bert_tokenizer)
try:
    model.load_state_dict(torch.load('./model_state_dict.pth'))
    print('Loaded previous model state successfully!')
except:
    print('Starting fresh! Previous model state dict load unsuccessful')


Loaded previous model state successfully!


In [ ]:
model = model.to(device)

In [ ]:
torch.save(model.state_dict(), './model_'+col_name+'_'+str(datetime.datetime.now())+'.pth')

In [ ]:
test_submission_data = pd.read_csv('./final_test3_unpreprocessed.csv')
test_submission_dataset=SubmissionDataset(test_submission_data, './test_images', img_transformations, bert_tokenizer, vocab)
test_submission_dataloader=torch.utils.data.DataLoader(test_submission_dataset, batch_size=4, collate_fn=collate_function_for_submission)

In [ ]:
len(test_submission_data)


In [ ]:
predictions, tweet_ids = model_predict(test_submission_dataloader, model, chosen_criteria, 1)

In [ ]:
for i in range(len(predictions)):
    predictions[i]=(predictions[i][0])

In [ ]:
predictions = np.array(predictions).reshape(-1, 1)

In [ ]:
tids = []
for i in range(len(tweet_ids)):
    tids+=[[str(tweet_ids[i][0])]]
tids_arr = np.array(tids)
tids_arr.shape

In [ ]:
# TweetIds[0]

In [ ]:
# tweet_ids = np.array(tweet_ids).reshape(-1, 1)

In [ ]:
# for i in range(tweet_ids.shape[0]):
#     tweet_ids[i][0]=str(tweet_ids[i][0])

In [ ]:
# type(tweet_ids[0][0])

In [ ]:
submit_df = pd.DataFrame(np.concatenate((tids_arr, predictions), axis=1), columns=['TweetId', col_name])

In [ ]:
submit_df[submit_df[col_name]==0]

In [ ]:
submit_df.to_csv(col_name+' '+str(datetime.datetime.now())+'.csv')

In [ ]:
str(datetime.datetime.now())